In [1]:
import sys 
sys.path

['/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python36.zip',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/lib-dynload',
 '',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/site-packages',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/site-packages/IPython/extensions',
 '/home/xuenhui/.ipython']

In [2]:
#添加导包路径
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python36.zip')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/lib-dynload')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/site-packages')

This solution uses only machine learning methods to "blind guess" from file name information.

- Specifically, the name of the video file is extracted and divided into segments, and a total of 5 features are extracted, and after feature encoding, they are sent to XGB for training.

- Best score on test1:

  - result using original parameters: 0.708238；0.682046

  - Results after using grid search: 0.7501；0.7179 (Final Results)
  

Subsequent optimization ideas:
- [x] Feature stitching with fea340 or 480
- [ ] Extract the training data of the same type as test1 in the training set (same forgery method), and train test1 separately



In [3]:
import pandas as pd
import numpy as np
train = pd.read_csv('data/label/train_set.csv')
test1 = pd.read_csv('data/label/test_set1.txt', names=['file'])
test2 = pd.read_csv('data/label/test_set2.txt', names=['file'])
test3 = pd.read_csv('data/label/test_set3.txt', names=['file'])

In [4]:
features = train.drop(['mos'],axis = 1)
# labels = train['mos']
labels = np.array(list(train['mos']), dtype=np.float)


In [5]:
print(type(labels))
print(type(features))

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [6]:
features['file_name'] = features['file'].apply(lambda x: x.split("/")[0])
features['id1'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
features['id2'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
features['id3'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
features['man'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

In [7]:
test1_features = test1
test1_features['file_name'] = test1_features['file'].apply(lambda x: x.split("/")[0])
test1_features['id1'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test1_features['id2'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test1_features['id3'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test1_features['man'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

In [8]:
test2_features = test2
test2_features['file_name'] = test2_features['file'].apply(lambda x: x.split("/")[0])
test2_features['id1'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test2_features['id2'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test2_features['id3'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test2_features['man'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

test3_features = test3
test3_features['file_name'] = test3_features['file'].apply(lambda x: x.split("/")[0])
test3_features['id1'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test3_features['id2'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test3_features['id3'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test3_features['man'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

In [9]:
# Make a tag before merging to facilitate splitting
features['tag'] = 'train'
test1_features['tag'] = 'test1'
test2_features['tag'] = 'test2'
test3_features['tag'] = 'test3'



from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# It is necessary to combine the training set and the test set, and then do LabelEncoder uniformly
all_df = pd.concat([features, test1_features, test2_features, test3_features])
all_df['file'] = all_df['file'].astype('str')
all_df['file'] = le.fit_transform(all_df['file'])
# Perform LabelEncoder on id1
all_df['id1'] = all_df['id1'].astype('str')
all_df['id1'] = le.fit_transform(all_df['id1'])

all_df['id2'] = all_df['id2'].astype('str')
all_df['id2'] = le.fit_transform(all_df['id2'])

all_df['id3'] = all_df['id3'].astype('str')
all_df['id3'] = le.fit_transform(all_df['id3'])

all_df['file_name'] = all_df['file_name'].astype('str')
all_df['file_name'] = le.fit_transform(all_df['file_name'])

all_df['man'] = all_df['man'].astype('str')
all_df['man'] = le.fit_transform(all_df['man'])



# Split the encoded features into train, test1, test2, test3
features['file'] = all_df[all_df['tag']=='train']['file']
features['file_name'] = all_df[all_df['tag']=='train']['file_name']
features['id1'] = all_df[all_df['tag']=='train']['id1']
features['id2'] = all_df[all_df['tag']=='train']['id2']
features['id3'] = all_df[all_df['tag']=='train']['id3']
features['man'] = all_df[all_df['tag']=='train']['man']
features = features.drop(['tag'],axis = 1)
features = features.drop(['file'],axis = 1)
features


test1_features['file'] = all_df[all_df['tag']=='test1']['file']
test1_features['file_name'] = all_df[all_df['tag']=='test1']['file_name']
test1_features['id1'] = all_df[all_df['tag']=='test1']['id1']
test1_features['id2'] = all_df[all_df['tag']=='test1']['id2']
test1_features['id3'] = all_df[all_df['tag']=='test1']['id3']
test1_features['man'] = all_df[all_df['tag']=='test1']['man']
test1_features = test1_features.drop(['tag'],axis = 1)
test1_features = test1_features.drop(['file'],axis = 1)


test2_features['file'] = all_df[all_df['tag']=='test2']['file']
test2_features['file_name'] = all_df[all_df['tag']=='test2']['file_name']
test2_features['id1'] = all_df[all_df['tag']=='test2']['id1']
test2_features['id2'] = all_df[all_df['tag']=='test2']['id2']
test2_features['id3'] = all_df[all_df['tag']=='test2']['id3']
test2_features['man'] = all_df[all_df['tag']=='test2']['man']
test2_features = test2_features.drop(['tag'],axis = 1)
test2_features = test2_features.drop(['file'],axis = 1)


test3_features['file'] = all_df[all_df['tag']=='test3']['file']
test3_features['file_name'] = all_df[all_df['tag']=='test3']['file_name']
test3_features['id1'] = all_df[all_df['tag']=='test3']['id1']
test3_features['id2'] = all_df[all_df['tag']=='test3']['id2']
test3_features['id3'] = all_df[all_df['tag']=='test3']['id3']
test3_features['man'] = all_df[all_df['tag']=='test3']['man']
test3_features = test3_features.drop(['tag'],axis = 1)
test3_features = test3_features.drop(['file'],axis = 1)

In [10]:
features = features.values
print(type(features))
print(features)

<class 'numpy.ndarray'>
[[ 0  0  0  7  0]
 [ 0  0  0  7  2]
 [ 0  0  0  7  3]
 ...
 [ 2 19  1 12 30]
 [ 2 19  1 12 31]
 [ 2 19  1 12 32]]


In [11]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=1)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, max_depth=6,
         subsample=0.6, colsample_bytree=0.8, learning_rate=0.05, random_state=0)
model.fit(X_train, y_train)
model.score(X_test, y_test)


0.7386349841110025

In [12]:
import numpy as np
from sklearn.metrics import confusion_matrix, mean_squared_error
pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse

0.4170655737262433

In [15]:
# regression cross validation
rng = np.random.RandomState(123)
kf = KFold(n_splits=3, shuffle=True, random_state=rng)
print("在3折数据上的交叉验证")

for train_index, test_index in kf.split(features):
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror',
                                 n_estimators=300, 
                                 max_depth=6,
                                 subsample=0.6,
                                 colsample_bytree=0.8,learning_rate=0.1,random_state=0)
    xgb_model.fit(features[train_index],labels[train_index])                             
    predictions = xgb_model.predict(features[test_index])
    actuals = labels[test_index]
    print("均方根误差:")
    print(np.sqrt(mean_squared_error(actuals, predictions)))
    print('拟合优度')
    print(xgb_model.score(features[test_index],labels[test_index]))

    
    
# Regression grid search for optimal hyperparameters
model = xgb.XGBRegressor(objective='reg:squarederror',
                         subsample=0.6, 
                         colsample_bytree=0.8, 
                         random_state=0,nthread=8)
param_dict = {'max_depth': [5,6,7,8],
              'n_estimators': [200,150,250],
              'learning_rate':[0.05,0.1,0.2]}
              
clf = GridSearchCV(model, param_dict, cv=10, verbose=1 , scoring='r2')
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)


在3折数据上的交叉验证
均方根误差:
0.4404700223612908
拟合优度
0.6704943023231531
均方根误差:
0.4058233239969286
拟合优度
0.7354703916906666
均方根误差:
0.429908598706019
拟合优度
0.6903203544956465
Fitting 10 folds for each of 36 candidates, totalling 360 fits
0.7091687456866139
{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 150}


In [16]:
# predict
test1_pred = clf.predict(test1_features)
test2_pred = clf.predict(test2_features)
test3_pred = clf.predict(test3_features)

In [17]:
ttest1 = pd.read_csv('data/label/test_set1.txt', names=['file'])
ttest2 = pd.read_csv('data/label/test_set2.txt', names=['file'])
ttest3 = pd.read_csv('data/label/test_set3.txt', names=['file'])
#save result
import os
test1_res = pd.DataFrame(ttest1['file'])
test1_res['mos'] = test1_pred
test1_res.to_csv(os.path.join(r'./pred_3(aistudio)/grid_search/', 'Test1_preds.txt'), index=None, header=None)